In [1]:
import pandas as pd
import tarfile
import csv
from nltk.tokenize import word_tokenize
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# META DATA

In [2]:
meta_speaker = pd.read_csv('speaker.tsv',  sep='\t')
meta_session = pd.read_csv('meta.tsv',  sep='\t')

In [3]:
meta_session.head(2)

,basename,date,num_speeches,topic,pressrelease,outcome,year,month,day
0,UNSC_1995_SPV.3486,6 January 1995,1,Bosnia and Herzegovina,NaN,http://www.un.org/en/ga/search/view_doc.asp?sy...,1995,1,6
1,UNSC_1995_SPV.3487,12 January 1995,40,Federal Republic of Yugoslavia (Serbia and Mon...,NaN,http://www.un.org/en/ga/search/view_doc.asp?sy...,1995,1,12


In [4]:
meta_speaker.head(2)

,speech,country,speaker,participanttype,role_in_un,spv,basename,topic,date,year,...,filename,types,tokens,sentences,topic2,subtopic,agenda_item1,agenda_item2,agenda_item3,decision
0,1,Argentina,Mr. Cardenas,The President,NaN,3486,UNSC_1995_SPV.3486,Bosnia and Herzegovina,6 January 1995,1995,...,UNSC_1995_SPV.3486_spch001.txt,305,862,29,Items relating to the situation in the former ...,Bosnia and Herzegovina,Europe,Former Yugoslavia,Bosnia and Herzegovina,PRST
1,1,Argentina,Mr. Cardenas,The President,NaN,3487,UNSC_1995_SPV.3487,Federal Republic of Yugoslavia (Serbia and Mon...,12 January 1995,1995,...,UNSC_1995_SPV.3487_spch001.txt,203,572,17,Items relating to the situation in the former ...,Bosnia and Herzegovina,Europe,Former Yugoslavia,Bosnia and Herzegovina,NaN


In [5]:
meta_session.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5748 entries, 0 to 5747
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   basename      5748 non-null   object
 1   date          5748 non-null   object
 2   num_speeches  5748 non-null   int64 
 3   topic         5748 non-null   object
 4   pressrelease  5087 non-null   object
 5   outcome       3038 non-null   object
 6   year          5748 non-null   int64 
 7   month         5748 non-null   int64 
 8   day           5748 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 404.3+ KB


In [6]:
meta_speaker.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82165 entries, 0 to 82164
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   speech           82165 non-null  int64 
 1   country          82165 non-null  object
 2   speaker          81631 non-null  object
 3   participanttype  82165 non-null  object
 4   role_in_un       2697 non-null   object
 5   spv              82165 non-null  object
 6   basename         82165 non-null  object
 7   topic            82165 non-null  object
 8   date             82165 non-null  object
 9   year             82165 non-null  int64 
 10  month            82165 non-null  int64 
 11  day              82165 non-null  int64 
 12  filename         82165 non-null  object
 13  types            82165 non-null  int64 
 14  tokens           82165 non-null  int64 
 15  sentences        82165 non-null  int64 
 16  topic2           82165 non-null  object
 17  subtopic         81901 non-null

# Importing speeches data

In [7]:
data = {
    "basename": [],
    "speech_id": [],
    "spv": [],
    "speech": []
}

with tarfile.open('speeches.tar') as tar_file:
    
    files = tar_file.getmembers()
    
    for file in files:
        file_name = file.name

        basename = file_name[:int(file_name.find('_spch'))]
        spv = basename[-4:]
        speech_id = int(file_name[int(file_name.find('_spch')):].strip('.txt').strip('_spch'))
        
        speech = str(tar_file.extractfile(file).read())
        speech = speech[str(speech).find(':')+2:]

        data["basename"].append(basename)
        data["speech_id"].append(speech_id)
        data["spv"].append(spv)
        data["speech"].append(speech)
        
        extracted_file = tar_file.extractfile(file).read()

df_speeches = pd.DataFrame(data)
df_speeches["speech"] = df_speeches["speech"].str.replace(r'\n', ' ')
df_speeches.head(5)

,basename,speech_id,spv,speech
0,UNSC_1995_SPV.3486,1,3486,As this is the first meeting of the Security C...
1,UNSC_1995_SPV.3487,1,3487,I should like at the outset of the meeting to ...
2,UNSC_1995_SPV.3487,2,3487,"First, Mr. President, let me welcome you to yo..."
3,UNSC_1995_SPV.3487,3,3487,The next speaker is the representative of Croa...
4,UNSC_1995_SPV.3487,4,3487,"Allow me, Sir, to congratulate you on your ass..."


In [8]:
int(speech_id)

18

In [9]:
meta_session.columns

Index(['basename', 'date', 'num_speeches', 'topic', 'pressrelease', 'outcome',
       'year', 'month', 'day'],
      dtype='object')

In [10]:
meta_speaker.columns

Index(['speech', 'country', 'speaker', 'participanttype', 'role_in_un', 'spv',
       'basename', 'topic', 'date', 'year', 'month', 'day', 'filename',
       'types', 'tokens', 'sentences', 'topic2', 'subtopic', 'agenda_item1',
       'agenda_item2', 'agenda_item3', 'decision'],
      dtype='object')

In [11]:
df_speeches['basename']

0        UNSC_1995_SPV.3486
1        UNSC_1995_SPV.3487
2        UNSC_1995_SPV.3487
3        UNSC_1995_SPV.3487
4        UNSC_1995_SPV.3487
                ...        
82160    UNSC_2020_SPV.8778
82161    UNSC_2020_SPV.8778
82162    UNSC_2020_SPV.8778
82163    UNSC_2020_SPV.8778
82164    UNSC_2020_SPV.8778
Name: basename, Length: 82165, dtype: object

# UNSC Consolidated csv file

In [12]:
df = pd.merge(left=df_speeches, 
                     right=meta_session[['basename', 'date','topic','year', 'month', 'day']], 
                     how='inner', on='basename')

df.head(2)

,basename,speech_id,spv,speech,date,topic,year,month,day
0,UNSC_1995_SPV.3486,1,3486,As this is the first meeting of the Security C...,6 January 1995,Bosnia and Herzegovina,1995,1,6
1,UNSC_1995_SPV.3487,1,3487,I should like at the outset of the meeting to ...,12 January 1995,Federal Republic of Yugoslavia (Serbia and Mon...,1995,1,12


In [13]:
meta_speaker.rename(columns={"speech": "speech_id"}, inplace=True)

df = pd.merge(left=df,
         right=meta_speaker[['speech_id', 'country', 'speaker', 'basename']],
         on=['basename', 'speech_id'],
         how='inner')

df.head(2)

,basename,speech_id,spv,speech,date,topic,year,month,day,country,speaker
0,UNSC_1995_SPV.3486,1,3486,As this is the first meeting of the Security C...,6 January 1995,Bosnia and Herzegovina,1995,1,6,Argentina,Mr. Cardenas
1,UNSC_1995_SPV.3487,1,3487,I should like at the outset of the meeting to ...,12 January 1995,Federal Republic of Yugoslavia (Serbia and Mon...,1995,1,12,Argentina,Mr. Cardenas


In [14]:
df['full_date'] = pd.to_datetime(df[['year', 'month', 'day']])
df['month_year'] = pd.to_datetime(df[['year', 'month', 'day']]).dt.strftime('%Y/%m')

# Tokenizing

In [15]:
def tokenize_text(text):
    tokens = word_tokenize(text)
    return tokens

df['speech_tokenized'] = df['speech'].apply(tokenize_text)

In [16]:
df.columns

Index(['basename', 'speech_id', 'spv', 'speech', 'date', 'topic', 'year',
       'month', 'day', 'country', 'speaker', 'full_date', 'month_year',
       'speech_tokenized'],
      dtype='object')

In [17]:
df[['country', 'full_date', 'speech', 'speech_tokenized', 'topic']].to_csv('UNSC.csv', index=False)